In [ ]:
# Following to be uncommented on first run ...

from helper import get_connected
from ib_insync import *
util.startLoop()
ib = get_connected('nse', 'live') # close the connection and delete it final function

In [ ]:
%%time
# WEEKEND GENERATION PROCESS - UNDERLYINGS FIRST
# ______________________________________________

# testing pickle nses function

from nse_func import p_nses
df = p_nses(ib)
df

In [ ]:
%%time
# WEEKEND GENERATION PROCESS - OPTIONS SECOND
# ______________________________________________

# testing pickle nse options function

import pandas as pd
from nse_func import p_nseopts

fspath = '../data/nse/'
df = pd.read_pickle(fspath+'_lot_margin.pickle')
[p_nseopts(ib=ib,undContract=u, undPrice=p, lotsize=z, margin=m) for u, p, z, m in zip(df.contract, df.undPrice, df.lot, df.margin)]

In [ ]:
# EVERYDAY UPDATE PROCESS (FIRST PART)
# ___________________________________

# testing nse underlying update function

import pandas as pd
fspath = '../data/nse/'
dfu = pd.read_pickle(fspath+'_lot_margin.pickle')
dfu

In [ ]:
dfu.loc[:, ['margin', 'undPrice']] = 0 # zerorize margin and undPrice to test
dfu

In [ ]:
%%time

from nse_func import upd_nses
upd_nses(ib, dfu)

In [ ]:
%%time

# EVERYDAY UPDATE PROCESS (SECOND PART - OPT PRICE)
# ________________________________________________

# Option price update
# Note: If the DataFrame given is a 'target' one, it will be every minute

import pandas as pd
fspath = '../data/nse/'

from os import listdir

from helper import upd_opt

blk = 50
fs = listdir(fspath)

# collect the options
optsList = [f for f in fs if f[-3:] == 'pkl']
dfopts = pd.concat([pd.read_pickle(fspath+f) 
                 for f in optsList], axis=0, sort=True).reset_index(drop=True)

# all options with price updated
df = upd_opt(ib, dfopts)

In [ ]:
df.head()

In [ ]:
# EVERYDAY UPDATE PROCESS (THIRD PART - REMAINING QUANTITIES PRICE)
# _________________________________________________________________

exchange = 'NSE'
nse_assignment_limit = 1500000

# get the list of underlying contracts and dictionary of lots
qlm = get_nses(ib)
c_dict = qlm[0]       # {symbol: contract}
lots_dict = qlm[1]    # {symbol: lotsize}
margin_dict = qlm[2]  # {symbol: margin}

undContracts = [v for k, v in c_dict.items()]

tickers = ib.reqTickers(*undContracts)
undPrices = {t.contract.symbol: t.marketPrice() for t in tickers} # {symbol: undPrice}

df_und = \
    pd.DataFrame.from_dict(lots_dict, orient='index', columns=['lotsize']).\
    join(pd.DataFrame.from_dict(undPrices, orient='index', columns=['undPrice'])).\
    join(pd.DataFrame.from_dict(c_dict, orient='index', columns=['undContract'])).dropna()

df_und = df_und.assign(remq=(nse_assignment_limit/(df_und.lotsize*df_und.undPrice)).astype('int')) # remaining quantities in entire nse

# from portfolio
#_______________

p = util.df(ib.portfolio()) # portfolio table

# extract option contract info from portfolio table
dfp = pd.concat([p, util.df([c for c in p.contract])[util.df([c for c in p.contract]).columns[:7]]], axis=1).iloc[:, 1:]
dfp = dfp.rename(columns={'lastTradeDateOrContractMonth': 'expiration'})

# get the underlying's margins, lots, prices and positions
pos_dict = dfp.groupby('symbol')['position'].sum().to_dict()
p_syms = {s for s in dfp.symbol}
p_undc = {s: ib.qualifyContracts(Stock(s, exchange)) for s in p_syms}   # {symbol: contracts} dictionary
p_undMargins = {u: margin_dict[u] for u in p_syms} # {symbol: undMargin} dictionary
p_undLots = {u: lots_dict[u] for u in p_syms}      # {symbol: lots} dictionary
p_undPrices = {u: undPrices[u] for u in p_syms}    #{symbol: undPrice} dictionary

dfp1 = pd.DataFrame.from_dict(p_undc, orient='index', columns=['contract']). \
    join(pd.DataFrame.from_dict(p_undMargins, orient='index', columns=['undmargin'])). \
    join(pd.DataFrame.from_dict(p_undLots, orient = 'index', columns=['undLot'])). \
    join(pd.DataFrame.from_dict(p_undPrices, orient='index', columns=['undPrice'])). \
    join(pd.DataFrame.from_dict(pos_dict, orient='index', columns=['position']))

dfp1 = dfp1.assign(qty=(dfp1.position/dfp1.undLot).astype('int'))

# make the blacklist
#___________________
remqty_dict = pd.DataFrame(df_und.loc[dfp1.index].remq + dfp1.qty).to_dict()[0] # remq from portfolio
remqty_dict = {k:(v if v > 0 else 0) for k, v in remqty_dict.items()} # portfolio's remq with negative values removed
blacklist = [k for k, v in remqty_dict.items() if v <=0] # the blacklist
df_und.remq.update(pd.Series(remqty_dict)) # replace all underlying with remq of portfolio
remqty = df_und.remq.to_dict() # dictionary
return remqty